In [2]:
!pip install llama_index

In [3]:
!pip install llama-index-llms-huggingface

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

In [3]:
!mkdir data

In [5]:
# load documents
documents = SimpleDirectoryReader("./data/").load_data()

In [ ]:
documents

In [6]:
# setup prompts - stable to LM
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as accurately
as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}\n<|ASSISTANT|>")


In [7]:
import torch

In [8]:
# loading llm in the notebook
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    device_map="auto",
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    stopping_ids=[50278, 50279, 50277,1,0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
# load embedding model
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor

In [17]:
!pip show llama-index

Name: llama-index
Version: 0.14.5
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


In [9]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# from llama_index.core import VectorStoreIndex, ServiceContext
# service_context = ServiceContext.from_defaults(
#     chunk_size=1024,
#     llm=llm,
#     embed_model=embed_model
#     )


In [14]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024


In [15]:
index = VectorStoreIndex.from_documents(documents)

In [16]:
query_engine = index.as_query_engine()

In [17]:
response = query_engine.query("What is attention ?")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [18]:
print(response)

Attention is a mechanism used in machine translation to retain and utilize all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence. The attention mechanism just addresses the issue of the encoder not being able to memorize the words coming at the beginning of the sentences, which leads to poor translation of the source sentence. During the decoding phase, the model creates an alignment between each time step of the decoder output and all of the encoder hidden state. We need to learn this alignment. Each output of the decoder can selectively pick out specific elements from the sequence to produce the output. This allows the model to focus and pay more attention to the relevant part of the input sequence.
